# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [1]:
# Base Functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# dataset = pd.read_csv('../data/flights/flightFinalTrain.csv')
# dataset = dataset.drop(['Unnamed: 0','tail_num'], axis=1)
# data = dataset.sample(n=100000, axis=0, random_state=42)
# dataset = pd.read_csv('../data/passengerTrain.csv')
# dataset2 = pd.read_csv('../data/flights/newflightTrain.csv')
data = pd.read_csv('../data/training_set/final_to_train.csv')

In [2]:
#data = pd.get_dummies(data,prefix=['mkt_carrier'], columns = ['mkt_carrier'], drop_first=True)

In [15]:
data = data.drop(['Unnamed: 0', 'origin_city', 'dest_city_name'],axis=1)


KeyError: "['Unnamed: 0' 'origin_city'] not found in axis"

In [16]:
data = data.drop(['dest_city_name'],axis=1)

In [17]:
data['origin_airport_id'].value_counts()

10397    726554
13930    476423
11292    433463
12892    411909
11298    407176
14747    310412
14771    307156
14107    305359
11057    301068
12266    238675
Name: origin_airport_id, dtype: int64

In [18]:
sample = data.loc[data['origin_airport_id'] == 12266]

In [19]:
sample.columns

Index(['origin_airport_id', 'dest_airport_id', 'day', 'month', 'year',
       'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'carrier_AS', 'carrier_B6',
       'carrier_DL', 'carrier_F9', 'carrier_G4', 'carrier_HA', 'carrier_NK',
       'carrier_UA', 'carrier_VX', 'carrier_WN', 'arr_delay', 'windspeeds',
       'visibilities', 'precipitations', 'weather_Blowing_snow',
       'weather_Clear', 'weather_Cloudy', 'weather_Fog',
       'weather_Freezing_fog', 'weather_Heavy_freezing_drizzle',
       'weather_Heavy_rain', 'weather_Heavy_rain_at_times',
       'weather_Heavy_snow', 'weather_Ice_pellets', 'weather_Light_drizzle',
       'weather_Light_freezing_rain', 'weather_Light_rain',
       'weather_Light_rain_shower', 'weather_Light_sleet',
       'weather_Light_sleet_showers', 'weather_Light_snow',
       'weather_Light_snow_showers', 'weather_Mist',
    

In [20]:
X = sample.drop(['arr_delay'], axis=1)
y = sample['arr_delay']


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [22]:
# For classification
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)




from sklearn.metrics import mean_squared_error as mse

### Regression

In [23]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


2.0948989211270555e-05

In [8]:
y_pred

array([152., 182., 152., ..., 152., 217., 140.])

In [9]:
y_test

2054    -23.0
14843    -4.0
15793    56.0
9521    -10.0
19066    -8.0
         ... 
11838   -20.0
7977     87.0
644      59.0
18730    11.0
11523    -5.0
Name: arr_delay, Length: 3972, dtype: float64

In [10]:
from sklearn.metrics import mean_squared_error as mse

### Decision Tree

In [24]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

In [25]:
mse(y_test,y_pred)

4469.133717398136

### Random Forest

In [26]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [27]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [29]:
from sklearn.metrics import r2_score
mse(y_test, y_pred)
# r2_score(y_test, y_pred)

2514.766068712685

### Multiple Linear Regression

In [30]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [31]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)

In [33]:
from sklearn.metrics import r2_score
mse(y_test, y_pred)
# r2_score(y_test, y_pred)

2391.5024920338333

### Support Vector Regression

In [19]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

In [20]:
y_pred = sc.inverse_transform(regressor.predict(sc.transform(X_test)))
np.set_printoptions(precision=2)

ValueError: operands could not be broadcast together with shapes (3972,) (71,) (3972,) 

In [ ]:
mse(y_test, y_pred)

### Ensemble Methods

In [34]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 3.3 MB/s eta 0:00:01


In [35]:
import xgboost as xgb

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/scm/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/scm/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [7]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile
X, y = load_digits(return_X_y=True)
X_new = SelectKBest('SelectPercentile', k=10).fit_transform(X, y)
X_new.shape

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.